In [5]:
import nbimporter
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Cm
import csv
from PIL import Image
from io import BytesIO
from aux_calculations import convert_to_float,conv_vel_a_m


#extraer los datos de las tablas
def extract_patient_info(doc)->dict:
    '''
    Toma un archivo docx producido por un dispositivo vinno.
    Los datos del paciente son la primera tabla, la recorre y extrar el texto
    lo guarda en un diccionario
    '''
    data = {}
    #itero sobre las tablas del documento
    for index_table,table in enumerate(doc.tables):
        #tabla 1 es la que contiene estos datos
        if index_table==1:
            for row_index, row in enumerate(table.rows):
                if row_index>0:
                    for cell in row.cells:
                        if cell.text.lower().strip not in data:  
                            key=cell.text.split(':')[0].strip()
                            value=cell.text.split(':')[1].strip().replace('  ',' ')
                            data[key]=value            
    return data
##########################

def read_csv(csv_path: str) -> list:
    """Reads the CSV file and returns a list of rows as dictionaries."""
    with open(csv_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        data = list(csv_reader)
    return data



def find_measurement_limit(data: list) -> int:
    """Finds the row index that marks the start of measurements."""
    for index, row in enumerate(data):
        if 'Lucho Card' in row.values() or 'card2'in row.values():
            return index
    return None


def extract_row(row):

    if (
            '2' in row 
            and row['2'] is not None
            and row['2']!='' 
            and row['2'].replace('-','').replace('.','').isnumeric()

        ):
            value=[row['1'],row['2']]

    else:
        value=row['1']
    return value



def extract_measurements_from_csv(data: list, measurements_list: list) -> dict:
    """Extracts measurements from the CSV data based on predefined measurement types."""
    limite=find_measurement_limit(data)
    extracted_data={}
    extract_next_row = False
    
    #extraigo los datos
    for index,row in enumerate(data):
        #genero el key con el primer valor de la row para el diccionario

        key=row.get('ï»¿Name','').strip().replace("'",'')

        if not key:
            continue    

        if extract_next_row:
            extracted_data[vel_grad]=extract_row(row)
            extract_next_row = False  # Reset the flag    
            
        if key in measurements_list:
            # Check if the element exists in the current row
            extract_next_row = True
            # Set flag to extract the next row
            vel_grad=key+'_grad'


        extracted_data[key]=extract_row(row)
        #cheque
       # if 'Carotid'not in str(path)
        #lucho card marca el inicio de los calculo, asi que con ese tomo el index
        #si el indice es mayor al limite estamos en la segunda tabla. siempre sigue un patron donde el ante penultivo
        #elemento es el key que requiero y el valor esta en unir
        if limite is not None and index>limite+1:
            #controlo el len de keys y v para modificar que variable sera key
            if len(row.keys())==7:
                k=-4
                v=k+1
            elif len(row.keys())==6:
                k=-3
                v=k+1
            elif len(row.keys())==5:
                k=-2
                v=k+1
            if row[list(row.keys())[k]] is not None:
                key_2=row[list(row.keys())[k]].strip().replace("'",'')
                value_2=row[list(row.keys())[v]]

                #chequeo que valores 2 sea lista para no pner simbolos que compliquen la renderizacion
                if isinstance(value_2,list):
                    extracted_data[key_2]=value_2[0]
                else: extracted_data[key_2]=value_2
    extracted_data=convert_to_float(extracted_data)
    extracted_data=conv_vel_a_m(extracted_data)
    return extracted_data             


def extract_patient_measurements(path)-> dict:
    '''
    toma el path del docx, lo cambia a un path terminado en .csv
    Extrae las mediciones del archivo csv del directorio de un paciente, generado por un dispositivo vinno
    '''
    #convierto path de doc en csv
    csv_path = path.with_suffix('.csv').as_posix().replace('Report.V3', 'Worksheet')
    
    #lista para convertir a gradientes    
    measurements_list = ['RVOT Vmax', 'AV Vmax', 'LVOT Vmax', 'TR Vmax', 'MV Vel E', 'MV Vel A']
    #la siguientes fila de las velocidades estan los gradientes
    
 
    try:
        csv_data = read_csv(csv_path)
        measurement_data=extract_measurements_from_csv(csv_data,measurements_list)
    except Exception as e:
        print(f"Error: {e}, unable to process the file")
        
    #genero el limite de la tabla de mediciones

    return measurement_data

#########################

def image_extractor(doc, template,tipo,image_width=Cm(8), image_height=Cm(5.36))->dict:
    '''
    extrae las imagenes del reporte docx del dispositivo vinno y devuelve un diccionario con objetos inlineimage
    requiere el doc: documento a extraer, template: template donde se renderizaran
    Se establecen medidas habituales de 5.36x8, excepto para el mapa polar del stress que es 8.22 x 16.23 y 6.39 x 16.23
    
    
    '''
    image_dict={}
    #extraer imagenes
    for rel in doc.part.rels:
        rel_obj = doc.part.rels[rel]
        if 'image' in rel_obj.reltype:
            image_data = rel_obj.target_part.blob
            image=Image.open(BytesIO(image_data))
            compressed_image = BytesIO()
            if image.format != 'JPEG':
                image = image.convert("RGB")  # Convert to RGB for JPEG
            image.save(compressed_image, format='JPEG', quality=85)  # Adjust quality to reduce size
            compressed_image.seek(0)
            
            target= rel_obj.target_ref.split('.')[0].replace(r'media/','')
            #si el tipo de template es stres las primeras dos imagenes tiene un tamano distinto del resto
            
            if tipo=='stress' and int(target.replace('image','')) in [1,2]:
    
                if target.replace('image','')=='1':
                    image_dict[target]=InlineImage(template,
                                          compressed_image,#BytesIO(image_data),
                                          width=Cm(16.23), height=Cm(8.22)
                                         )
                    
                else:
                        image_dict[target] = InlineImage(template,
                                          compressed_image,#BytesIO(image_data),
                                          width=Cm(16.23), height=Cm(6.39)
                                         )
            else:          
                #default tamano de imagenes
                image_dict[target] = InlineImage(template,
                                                 compressed_image,#BytesIO(image_data),
                                                 width=image_width,
                                                 height=image_height,

                                                )

    
    key=list(image_dict.keys())
    key = sorted(key, key=lambda image_name: int(image_name.replace('image', '')))
    image_dict={i: image_dict[i] for i in key}
    image_dict={'image':[{'key':k,'image':v} for k,v in image_dict.items()]}
    
    
    return image_dict

def mot_extractor(doc)->dict:
    
    '''
    Extrae las puntaciones y los nombres de los segmentos en los score de motilidad del ecostress
    '''
    table=doc.tables[2].rows[1].cells[0].tables[0]
    mot={}
    for index_r, row in enumerate(table.rows):
        #la fila 1 contiene la primera info,la ultima info en la 17 (apex)
        if 1 <= index_r <= 17:
            key = None
            values = []
            for index_c, cell in enumerate(row.cells):
                #las primeras cuatro celdas son segment ID
                #la 5 celda es el nombre del segmento
                #la 6 es baseline, 7 peak, 8 recovery 
                if index_c == 5:
                    key = cell.text.lower().replace(' ', '_')
                if 5 < index_c <= 8 and key:
                    values.append(cell.text)

            if key and values:  # Store the key-value pair only if both key and values exist
                mot[key] = values
                contex = {'mot': [{'key': k, 'motilidad': v} for k, v in mot.items()]}                                             
                                                                   
    return contex                                                              
